In [139]:
#%%
import pandas as pd
import numpy as np
import docx
import pickle
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer, WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
import textwrap
nltk.download('punkt')
sent_tokenizer=nltk.data.load('tokenizers/punkt/portuguese.pickle')
#npr =  pd.read_pickle('../arquivos_pickle/docs_df_tokens_originais.pkl')


[nltk_data] Downloading package punkt to /home/manuel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [140]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/manuel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [141]:
with open('data_split_by_clausese/autorizacoes1.txt') as f:
    text = f.read()
text


"AUTORIZAÇÕES \nAutorizações da Emissora. A presente Escritura de Emissão é celebrada com base nas deliberações tomadas na assembleia geral extraordinária de acionistas da Emissora, realizada em [•] de março de 2023 ('Aprovação Societária Emissora'), na qual foram deliberadas e aprovadas, dentre outras matérias: (i) a realização da Emissão (conforme definido abaixo) e da Oferta Pública (conforme definido abaixo), bem como os seus termos e condições; (ii) a outorga da Cessão Fiduciária (conforme definido abaixo), em favor dos Debenturistas, por meio do Contrato de Cessão Fiduciária (conforme definido abaixo); (iii) a autorização expressa à diretoria da Emissora para praticar todos os atos, tomar todas as providências e adotar todas as medidas necessárias à formalização, efetivação e administração das deliberações tomadas na Aprovação Societária Emissora, bem como para assinar todos e quaisquer documentos relacionados à Emissão e à Oferta Pública, incluindo, mas não se limitando, a esta 

In [142]:
doc = docx.Document('data/Eletropaulo teste.docx')
texto = [para.text for para in doc.paragraphs]
texto = '\n'.join(texto)
texto

'\n\nINSTRUMENTO PARTICULAR DE ESCRITURA DA 27ª (VIGÉSIMA SÉTIMA) EMISSÃO DE DEBÊNTURES SIMPLES, NÃO CONVERSÍVEIS EM AÇÕES, DA ESPÉCIE QUIROGRAFÁRIA, EM SÉRIE ÚNICA, PARA DISTRIBUIÇÃO PÚBLICA COM ESFORÇOS RESTRITOS, DA ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO PAULO S.A. \n\nsdgfsdfsdfsdfsdfsdfdsfsdfsdffsdfsd\nasfasdasdsadasdsa\n\n\n\n\nentre \nELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO PAULO S.A. \ncomo Emissora \n\ne\n\n\nOLIVEIRA TRUST DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A.\n representando a comunhão dos titulares das debêntures objeto da presente Emissão \n\n\n\n\n\n \n\n\n\n_________________________\ndatada de\n[=] de [abril] de 2022\n_________________________\n\n\n\nINSTRUMENTO PARTICULAR DE ESCRITURA DA 27ª (VIGÉSIMA SÉTIMA) EMISSÃO DE DEBÊNTURES SIMPLES, NÃO CONVERSÍVEIS EM AÇÕES, DA ESPÉCIE QUIROGRAFÁRIA, EM SÉRIE ÚNICA, PARA DISTRIBUIÇÃO PÚBLICA COM ESFORÇOS RESTRITOS, DA ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO PAULO S.A. \nPelo presente instrumento 

In [143]:
df = pd.DataFrame({'text': [text]})

In [144]:
def wrap(x):
    return textwrap.fill(x, replace_whitespace=False, fix_sentence_endings=True)

In [145]:
print(wrap(df.iloc[0]['text']))

AUTORIZAÇÕES 
Autorizações da Emissora.  A presente Escritura de
Emissão é celebrada com base nas deliberações tomadas na assembleia
geral extraordinária de acionistas da Emissora, realizada em [•] de
março de 2023 ('Aprovação Societária Emissora'), na qual foram
deliberadas e aprovadas, dentre outras matérias: (i) a realização da
Emissão (conforme definido abaixo) e da Oferta Pública (conforme
definido abaixo), bem como os seus termos e condições; (ii) a outorga
da Cessão Fiduciária (conforme definido abaixo), em favor dos
Debenturistas, por meio do Contrato de Cessão Fiduciária (conforme
definido abaixo); (iii) a autorização expressa à diretoria da Emissora
para praticar todos os atos, tomar todas as providências e adotar
todas as medidas necessárias à formalização, efetivação e
administração das deliberações tomadas na Aprovação Societária
Emissora, bem como para assinar todos e quaisquer documentos
relacionados à Emissão e à Oferta Pública, incluindo, mas não se
limitando, a esta E

In [146]:
sents = sent_tokenizer.tokenize(df.iloc[0]['text'])

In [147]:
sents

['AUTORIZAÇÕES \nAutorizações da Emissora.',
 "A presente Escritura de Emissão é celebrada com base nas deliberações tomadas na assembleia geral extraordinária de acionistas da Emissora, realizada em [•] de março de 2023 ('Aprovação Societária Emissora'), na qual foram deliberadas e aprovadas, dentre outras matérias: (i) a realização da Emissão (conforme definido abaixo) e da Oferta Pública (conforme definido abaixo), bem como os seus termos e condições; (ii) a outorga da Cessão Fiduciária (conforme definido abaixo), em favor dos Debenturistas, por meio do Contrato de Cessão Fiduciária (conforme definido abaixo); (iii) a autorização expressa à diretoria da Emissora para praticar todos os atos, tomar todas as providências e adotar todas as medidas necessárias à formalização, efetivação e administração das deliberações tomadas na Aprovação Societária Emissora, bem como para assinar todos e quaisquer documentos relacionados à Emissão e à Oferta Pública, incluindo, mas não se limitando, a 

In [148]:
featurizer = TfidfVectorizer(ngram_range=(1, 4), stop_words=stopwords.words('portuguese'), max_features=1000, norm='l1')

In [149]:
X = featurizer.fit_transform(sents)

In [150]:
def get_sentence_score(tfidf_row):
    x = tfidf_row[tfidf_row != 0]
    return x.mean()

In [151]:
scores = np.zeros(len(sents))
scores


array([0., 0., 0., 0.])

In [152]:
for i in range(len(sents)):
    score = get_sentence_score(X[i,:])
    scores[i] = score

In [153]:
scores

array([0.2       , 0.00185185, 0.33333333, 0.00283286])

In [154]:
sort_idx = np.argsort(-scores)

In [155]:
sort_idx

array([2, 0, 3, 1])

In [157]:
print("Sumario:")
for i in sort_idx[:4]:
    print(wrap("%.2f: %s" % (scores[i], sents[i])))

Sumario:
0.33: Autorização dos Acionistas.
0.20: AUTORIZAÇÕES 
Autorizações da Emissora.
0.00: A outorga da Alienação Fiduciária de Ações da Emissora (conforme
definido abaixo) por meio do Contrato de Alienação Fiduciária de Ações
(conforme definido abaixo) foi aprovada: (i) pela AEGEA Saneamento e
Participações S.A. ('AEGEA'), com base em autorização deliberada pela
reunião do conselho de administração da AEGEA, realizada em [•] de
março de 2023 ('Aprovação Societária AEGEA' e, em conjunto com a
Aprovação Societária Emissora, as 'Aprovações Societárias'); (ii) pelo
Kinea Equity Infra I Fundo de Investimento em Participações
Multiestratégia ('FIP Kinea'), com base na assembleia geral de
cotistas, realizada em [•] de fevereiro de 2023 em conformidade com o
seu regulamento ('Aprovação FIP Kinea'); (iii) pelo Perfin Mariner
Fundo de Investimento em Participações Multiestratégia ('FIP
Mariner'), com base no Termo de Encerramento de Consulta Formal datado
de 3 de fevereiro de 2023, em confo

In [1]:
import pickle

In [2]:
patter_path = 'arquivos_pickle/patterns.pkl'

In [3]:
patterns = pickle.load(open(patter_path, 'rb'))

In [4]:
patterns

[{'label': 'ORG', 'pattern': '2w energia s.a'},
 {'label': 'ORG', 'pattern': 'a  geradora  aluguel de maquinas s.a.'},
 {'label': 'ORG', 'pattern': 'a bodytech participacoes s.a.'},
 {'label': 'ORG', 'pattern': 'ab concessoes s.a.'},
 {'label': 'ORG', 'pattern': 'abaporu participacoes s/a'},
 {'label': 'ORG', 'pattern': 'abimex importacao e exportacao s/a'},
 {'label': 'ORG', 'pattern': 'acef s.a.'},
 {'label': 'ORG', 'pattern': 'ache laboratorios farmaceuticos s.a.'},
 {'label': 'ORG', 'pattern': 'acqio holding participacoes s.a.'},
 {'label': 'ORG', 'pattern': 'acucareira quata s/a'},
 {'label': 'ORG', 'pattern': 'adami s.a. - madeiras'},
 {'label': 'ORG', 'pattern': 'adecoagro vale do ivinhema s.a.'},
 {'label': 'ORG', 'pattern': 'adicional securitizadora s.a'},
 {'label': 'ORG', 'pattern': 'aegea saneamento e participacoes s/a'},
 {'label': 'ORG',
  'pattern': 'aeris industria e comercio de equipamentos para geracao de energia s.a.'},
 {'label': 'ORG', 'pattern': 'aeroportos brasil

In [9]:
orgs = [org['pattern'] for org in patterns]

In [10]:
orgs

['2w energia s.a',
 'a  geradora  aluguel de maquinas s.a.',
 'a bodytech participacoes s.a.',
 'ab concessoes s.a.',
 'abaporu participacoes s/a',
 'abimex importacao e exportacao s/a',
 'acef s.a.',
 'ache laboratorios farmaceuticos s.a.',
 'acqio holding participacoes s.a.',
 'acucareira quata s/a',
 'adami s.a. - madeiras',
 'adecoagro vale do ivinhema s.a.',
 'adicional securitizadora s.a',
 'aegea saneamento e participacoes s/a',
 'aeris industria e comercio de equipamentos para geracao de energia s.a.',
 'aeroportos brasil -  viracopos s.a.',
 'aes brasil energia s.a.',
 'aes tiete energia s/a',
 'aes tiete eolica s.a.',
 'aes tucano holding ii s.a',
 'aete - amazonia empresa transmissora de energia s.a.',
 'agasus s.a.',
 'ageo leste terminais e armazens gerais s.a.',
 'ageo norte terminais e armazens gerais s/a',
 'ageo terminais e armazens gerais s/a',
 'agora solucoes em tecnologia da informacao e comunicacao s a',
 'agricola alvorada s.a.',
 'agro industrias do vale do sao 